<a href="https://colab.research.google.com/github/babineax/Distributed-system/blob/main/DS_T2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
import bisect
import math

class ConsistentHashing:
    def __init__(self, num_servers, num_slots):
        self.num_servers = num_servers
        self.num_slots = num_slots
        self.num_virtual_servers = math.ceil(math.log2(num_slots))
        self.servers = {}
        self.ring = []

    def _hash_request(self, request_id):
        return (request_id + 2 * request_id**2 + 17) % self.num_slots

    def _hash_virtual_server(self, server_id, replica_id):
        return (server_id + replica_id + 2 * replica_id**2 + 25) % self.num_slots

    def _find_next_available_slot(self, slot):
        original_slot = slot
        i = 1
        while slot in self.servers:
            # Linear probing can be replaced by quadratic probing if needed.
            slot = (original_slot + i**2) % self.num_slots
            i += 1
        return slot

    def add_server(self, server_id):
        for replica_id in range(self.num_virtual_servers):
            virtual_server_hash = self._hash_virtual_server(server_id, replica_id)
            if virtual_server_hash in self.servers:
                virtual_server_hash = self._find_next_available_slot(virtual_server_hash)
            self.servers[virtual_server_hash] = server_id
            bisect.insort(self.ring, virtual_server_hash)

    def remove_server(self, server_id):
        keys_to_remove = [key for key, value in self.servers.items() if value == server_id]
        for key in keys_to_remove:
            self.servers.pop(key)
            self.ring.remove(key)

    def get_server(self, request_id):
        if not self.ring:
            return None
        request_hash = self._hash_request(request_id)
        idx = bisect.bisect(self.ring, request_hash)
        if idx == len(self.ring):
            idx = 0
        return self.servers[self.ring[idx]]

# Example usage
if __name__ == "__main__":
    num_servers = 3
    num_slots = 512

    ch = ConsistentHashing(num_servers, num_slots)

    # Adding servers
    for server_id in range(num_servers):
        ch.add_server(server_id)

    # Mapping requests to servers
    requests = [132574, 234567, 345678]
    for request_id in requests:
        server = ch.get_server(request_id)
        print(f"Request ID {request_id} is mapped to Server ID {server}")

    # Adding a new server
    new_server_id = 3
    ch.add_server(new_server_id)
    print(f"Added Server ID {new_server_id}")

    # Mapping requests to servers after adding a new server
    for request_id in requests:
        server = ch.get_server(request_id)
        print(f"Request ID {request_id} is now mapped to Server ID {server}")

    # Removing a server
    ch.remove_server(0)
    print("Removed Server ID 0")

    # Mapping requests to servers after removing a server
    for request_id in requests:
        server = ch.get_server(request_id)
        print(f"Request ID {request_id} is now mapped to Server ID {server}")


Request ID 132574 is mapped to Server ID 0
Request ID 234567 is mapped to Server ID 0
Request ID 345678 is mapped to Server ID 0
Added Server ID 3
Request ID 132574 is now mapped to Server ID 0
Request ID 234567 is now mapped to Server ID 0
Request ID 345678 is now mapped to Server ID 0
Removed Server ID 0
Request ID 132574 is now mapped to Server ID 1
Request ID 234567 is now mapped to Server ID 1
Request ID 345678 is now mapped to Server ID 1


Request ID 123456 is mapped to Server ID 2
Request ID 234567 is mapped to Server ID 0
Request ID 345678 is mapped to Server ID 0
Added Server ID 3
Request ID 123456 is now mapped to Server ID 2
Request ID 234567 is now mapped to Server ID 0
Request ID 345678 is now mapped to Server ID 0
Removed Server ID 0
Request ID 123456 is now mapped to Server ID 2
Request ID 234567 is now mapped to Server ID 1
Request ID 345678 is now mapped to Server ID 1
